In [7]:
from pathlib import Path
import subprocess
import os
import xml.etree.ElementTree as ET 
from xml.etree.ElementTree import Element
import json

# Simple Keywords Searching and Extraction

In [8]:
# take single xml document and parse to dictionary
def extract_ch_json(docPath, docType,docId):
    tree = ET.parse(docPath)
    root = tree.getroot()
    # initial dict for information storage
    outDict = {}
    outDict["id"] = docId
    outDict["type"] = docType
    outDict["title"] = ""
    outDict["chapters"] = {}
    # get document main title
    for elem in root.iter('{http://www.tei-c.org/ns/1.0}titleStmt'):
        outDict["title"] = elem[0].text
    # get chapters
    current_chapter = "" # current chapter name
    find_ch_flag = 0 # find chapter flag skip if not
    for div in root.iter("{http://www.tei-c.org/ns/1.0}div"): # iterate all div element
        for elem in div.iter("{http://www.tei-c.org/ns/1.0}head"): 
            if "chapter" in elem.text.lower(): # check head element if contains chapter
                current_chapter = elem.text
                outDict["chapters"][current_chapter] = current_chapter + "\n"
                find_ch_flag = 1
            elif find_ch_flag:
                outDict["chapters"][current_chapter] = outDict["chapters"][current_chapter]+elem.text+"\n"
        if find_ch_flag:       
            for elem in div.iter("{http://www.tei-c.org/ns/1.0}p"):
                outDict["chapters"][current_chapter] = outDict["chapters"][current_chapter]+elem.text+"\n"
    if find_ch_flag:
        return outDict # return dict if found
    else:
        return 0

## Sample Result 

In [9]:
docPath = "/mnt/6t/VT_ETDs/golden_standards/team16/processed/gorbid_fulltext/theses/17292/Bock_M_T_2017.tei.xml"
print(json.dumps(extract_ch_json(docPath,"thesis",17292), indent=4, sort_keys=True))

{
    "chapters": {
        "Chapter 2": "Chapter 2\nLiterature Review\nMotivating Works\nWhen approaching background research for this project, it was important to get a good grasp on the types of projects this framework would be supporting. To do this, two main areas were researched. First, a survey of the past two years of CS5604 (Information Retrieval) classes was conducted. The goal was to get a sense of the kinds of work teams are doing with the tweet collections. Second, a review of the IDEAL and GETAR projects was conducted. These two NSF-funded efforts have motivated much of the research work that has gone into creating, maintaining, and researching the digital library of tweets.\nCS 5604: Information Retrieval\nCS 5604 at Virginia Tech is a class which uses a project-based learning approach to teach students about information retrieval techniques. The project-based learning approach makes use of a single, semester-long project to guide students through the learning process \n

## Process documents

In [141]:
# process documents
pathlist = Path("/mnt/6t/VT_ETDs/golden_standards/team16/processed/gorbid_fulltext/theses/").glob('*')
extractedDoc = {}
for path in pathlist:
    dIn = str(path)
    docId = dIn.split("/")[-1]
    #print(docId)
    for subPath in path.glob('*'):
        if subPath.suffix == '.xml':
            docJson = extract_ch_json(subPath,"thesis",docId)
            if docJson != 0:
                with open('/mnt/6t/VT_ETDs/golden_standards/team16/processed/gorbid_fulltext/extracted/thesis/'+docId+'.json', 'w') as outfile:
                    json.dump(docJson, outfile)
            #print(subPath)


In [4]:
# test produced result 
test_json_data = open('/mnt/6t/VT_ETDs/golden_standards/team16/processed/gorbid_fulltext/extracted/thesis/17336.json').read()
test_data = json.loads(test_json_data)